In [1]:
from pathlib import Path
# import pickle
import warnings

import numpy as np
import pandas as pd
from pandarallel import pandarallel
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.metrics import pairwise_distances
from sqlalchemy import create_engine
from tqdm import tqdm

from src.models import cf, evaluate_model
from src.utilities import utilities

pandarallel.initialize()
tqdm.pandas()
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Pet_Supplies"
# MODEL_PATH = Path(f"models/ub_cf/ub_cf_{CATEGORY}.pkl")

# reproducibility checks
SEED = 42 
np.random.seed(42)

# load train dataset
train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A14CK12J7C7JRK,3.0,I purchased the Trilogy with hoping my two cat...,2011-01-12,purchase trilogy hop cat age interested yr old...
1,2,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2CR37UY3VR7BN,4.0,I bought the triliogy and have tested out all ...,2012-12-19,buy triliogy test dvd appear volume receive re...
2,3,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2A4COGL9VW2HY,4.0,My female kitty could care less about these vi...,2011-05-12,female kitty care video care little male dig a...
3,4,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2UBQA85NIGLHA,3.0,"If I had gotten just volume two, I would have ...",2012-03-05,volume star trilogy star read review know vol ...
4,5,B00005MF9U,LitterMaid LM900 Mega Self-Cleaning Litter Box,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",A2BH04B9G9LOYA,1.0,"First off, it seems that someone is spamming t...",2006-12-31,spamming review glow reviewer review amazon ba...
68865,111581,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A11J1FHCK5U06J,4.0,Now I know exactly where the trouble spots are...,2014-05-23,know exactly trouble spot sniffing guess invis...
68866,111585,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A18JF0T0GOCORW,4.0,I use this light to help me find stains when I...,2014-05-24,use light help stain carpet clean pre treat ca...
68867,111595,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A3GRPCW9DG427Z,5.0,We are owned by the 3 pickiest pooches in the ...,2013-07-27,pickiest pooch world love fool reject doggie t...
68868,111598,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A2X6TLAX3JEO1A,5.0,My highly allergic white boxer loves these tre...,2014-05-09,highly allergic white boxer love treat meat co...
68869,111602,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A9PG9ODPPP31N,5.0,Works great on my medium sized dog. She has ve...,2014-07-09,work great medium size dog coarse hair work gr...


In [4]:
# get user rating history
train_user_rating_history = train.groupby(["reviewerID"])["asin"].progress_apply(list)
print(train_user_rating_history)

100%|███████████████████████████████████████████| 19058/19058 [00:00<00:00, 67380.45it/s]

reviewerID
A04173782GDZSQ91AJ7OD                 [B0002AT464, B0002AT464, B00078ZK2S]
A042274212BJJVOBS4Q85                 [B001AT9B8M, B001E8LD3K, B00II7195M]
A0436342QLT4257JODYJ     [B000255NCI, B000255NCI, B000255O90, B000255O9...
A04795073FIBKY8GSLZYI                             [B000O39TDC, B000O39TE6]
A06658082A27F4VB5UG8E                             [B0006MU8WC, B000FS4OYA]
                                               ...                        
AZYJE40XW6MFG                                     [B0002IEYIE, B0002IEYIE]
AZZ56WF4X19G2                                                 [B0010P0YSW]
AZZNK89PXD006            [B0002DHV16, B0006N9D4A, B0018CIPS8, B001Q9EGK...
AZZV9PDNMCOZW                         [B0006N9LN8, B004PU7SBU, B00DIIKLCI]
AZZYW4YOE1B6E            [B0002ARQV4, B0002H3R2E, B0002H3R2E, B000MLG4K...
Name: asin, Length: 19058, dtype: object


# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [5]:
# loading test dataset
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [6]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,1,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A39QHP5WLON5HV,5.0,There are usually one or more of my cats watch...,2013-09-14,usually cat watch tv stay trouble dvd play lik...
1,104,B00005MF9V,LitterMaid Universal Cat Privacy Tent (LMT100),"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A366V0GCEPH5CX,5.0,My cats love it and so do I. I no longer have ...,2013-02-02,cat love longer cat litter fly floor litter fl...
2,133,B00005MF9T,LitterMaid LM500 Automated Litter Box,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",ALWWS8QBYN80B,1.0,I have one female cat that weighs under 10 pou...,2004-11-17,female cat weigh pound year old use everclean ...
3,153,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A3PVI3NE7OY1SP,5.0,I love these. They make the clean up so much e...,2013-09-26,love clean easy clean box manually use issue w...
4,154,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A2H83XMHUVDLJY,4.0,"I love this litter box. I do not use the lids,...",2014-06-26,love litter box use lid use receptacle tear cr...
41564,111601,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],AV34KNYW82YSS,4.0,Pulled lots of hair out of my Labs coat. Didn'...,2014-07-18,pulled lot hair labs coat think prove wrong co...
41565,111603,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1YMNTFLNDYQ1F,5.0,I have been trying to find a rubber bristle br...,2014-07-16,try rubber bristle brush persian year lose glo...
41566,111604,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1FQ3HRVXA4A5B,5.0,Great product to use on your pets knowing this...,2014-07-11,great product use pet know gentle rubber damag...
41567,111605,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A3OP6CI0XCRQXO,5.0,I bought a second one because I have two cats ...,2014-07-22,buy second cat american short hair buy brush m...
41568,111606,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A11LC938XF35XN,5.0,Our dogs love getting brushed with this. It m...,2014-07-17,dog love brush massage remove heavy undercoat ...


In [7]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [8]:
print(test_user_history)

                  reviewerID                                  asin
0      A04173782GDZSQ91AJ7OD              [B0090Z9AYS, B00CPDWT2M]
1      A042274212BJJVOBS4Q85              [B005AZ4M3Q, B00771WQIY]
2       A0436342QLT4257JODYJ  [B0018CDR68, B003SJTM8Q, B00474A3DY]
3      A04795073FIBKY8GSLZYI              [B001PKT30M, B005DGI2RY]
4      A06658082A27F4VB5UG8E              [B000TZ1TTM, B0019VUHH0]
...                      ...                                   ...
18993          AZYJE40XW6MFG              [B00HVAKJZS, B00IDZT294]
18994          AZZ56WF4X19G2                          [B004A7X218]
18995          AZZNK89PXD006  [B0002DHV16, B005BP8MQ8, B009RTX4SU]
18996          AZZV9PDNMCOZW              [B007EQL390, B00ISBWVT6]
18997          AZZYW4YOE1B6E  [B0002AQPA2, B0002AQPA2, B0002ARQV4]

[18998 rows x 2 columns]


## Instantiate Memory-based CF (User-based)

In [9]:
# instantiate model
ub_cf = cf.UserBasedCF()

In [10]:
%%time
# fit learning algorithm to training datab
ub_cf.fit(train, k_neighbours=50)

Generating user rating history...
Generating user-rating item rating (utility) matrix...
Generate user similarities matrix...
Generate k-neighbourhood of similar users...
CPU times: user 1min 36s, sys: 3.15 s, total: 1min 39s
Wall time: 1min 1s


In [11]:
# generate candidates items ranked by predicted ratings
candidate_items = ub_cf.test()

100%|██████████████████████████████████████████████| 19058/19058 [55:27<00:00,  5.73it/s]


## Save Model

In [12]:
# MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
# with open(MODEL_PATH, "wb") as f:
#     pickle.dump(ub_cf, f)

In [13]:
# # load model
# with open(MODEL_PATH, "rb") as f:
#     ub_cf = pickle.load(f)

## Loop through N = {10, 25, 30, 45}

In [14]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [15]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    # top_ns = get_top_n(candidate_items, train_user_rating_history, n)
    top_ns = ub_cf.get_top_n(n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model
                           .evaluate_recommendations(model_name = 'UB-CF',
                                                     top_ns = top_ns, 
                                                     user_rating_history = test_user_history,
                                                     item_popularity = item_popularity,
                                                     n = n,
                                                     mf_based = False))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The UB-CF has an average recall@10: 0.01088, average novelty@10: 0.91535
The UB-CF has an average recall@25: 0.02545, average novelty@25: 0.94071
The UB-CF has an average recall@30: 0.02813, average novelty@30: 0.94718
The UB-CF has an average recall@45: 0.03680, average novelty@45: 0.95868


# Evaluate N-Recommendations 

## N=10

In [16]:
top_ns_10 = n_recommendations[10][0]

In [17]:
utilities.retrieve_recommendations(train, top_ns_10)

For user: A3T87QAUUPTMZK:
Purchase History:
             asin                                              title
43366  B0010OSIHW  Zoo Med Eco Earth Compressed Coconut Fiber Sub...
45100  B00167VVP4   Zoo Med Eco Earth Loose Coconut Fiber Substra...
46998  B0019IJXD2       Zoo Med Reptile Fogger Terrarium Humidifier 

Recommending:

         asin                                              title
0  B0002ASM94  Nylabone Dura Chew Pooch Pacifier Chew Bone Va...
1  B00153RC9I  Halo Holistic Natural Dry Cat Food for Adult Cats
2  B00025640S  Tetra ReptoMin Floating Food Sticks for Aquati...
3  B0048Z73TY   DERMagic Shampoo Bar, 3.75 oz, Certified Organic
4  B0002AS5PU  Kaytee CritterTrail Fun-nels Tubes Accessories...
5  B009MKCJAI                                           KONG TUG
6  B0002ASC1M  Pureness Heavyweight Large Crock Pet Dish, 52-...
7  B0002AQ228  Prevue Hendryx Pet Products Wrought Iron Fligh...
8  B0002563KY       AquaClear Levelling Device for Power Filters
9  B0002DK6HM 

## N=25

In [18]:
top_ns_25 = n_recommendations[25][0]

In [19]:
utilities.retrieve_recommendations(train, top_ns_25)

For user: A1XEZIHQIUAOR1:
Purchase History:
             asin                                              title
1487   B000084E6V                      Nylabone Dental Dinosaur Chew
1727   B000084E6V                      Nylabone Dental Dinosaur Chew
28789  B0006VMN4O  Pioneer Pet SmartCat Peek-A-Prize Toy Box with...
37658  B000JZ1WSU                     SmartCat 3836 Tick Tock Teaser
42603  B000XZDV44  Hill'S Science Diet Kitten Savory Salmon Entre...
43867  B0012KCUOG  Whisker Lickin'S Soft &amp; Delicious Chicken ...

Recommending:

          asin                                              title
0   B0000AH3RP  Purina ONE Healthy Kitten Formula Dry Kitten Food
1   B00006H36X  Advantage Once-A-Month Topical Flea Treatment ...
2   B0002AT5AO           FAT CAT Crazy Circle Interactive Cat Toy
3   B00008DFTP  Purina Pro Plan Focus Small Breed Formula Adul...
4   B0000CEPDP               Hi-Tor Neo Diet For Cats 5.5-oz cans
5   B000HHLVA4                   Petmate Giant Litter Pan for

## N=30

In [20]:
top_ns_30 = n_recommendations[30][0]

In [21]:
utilities.retrieve_recommendations(train, top_ns_30)

For user: A89LQAXW1IY6S:
Purchase History:
             asin                                              title
12819  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
12825  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
38837  B000MD3NLS  MidWest Homes for Pets Snap'y Fit Stainless St...
42683  B000Y8UNAU  Pro Select Fleece Cat Perch Covers - Comfortab...
46465  B0018CJZ32                         SmartCat Corner Litter Box

Recommending:

          asin                                              title
0   B00153RC9I  Halo Holistic Natural Dry Cat Food for Adult Cats
1   B000I1M76Q  Midwest Cat Playpen | Cat Cage Includes 3 Adju...
2   B009MKCJAI                                           KONG TUG
3   B0048Z73TY   DERMagic Shampoo Bar, 3.75 oz, Certified Organic
4   B0002ASC1M  Pureness Heavyweight Large Crock Pet Dish, 52-...
5   B0002AS5PU  Kaytee CritterTrail Fun-nels Tubes Accessories...
6   B004YTZYV8  Bravo! Bonus Bites All Natural Freeze Dried Ve...

## N=45

In [22]:
top_ns_45 = n_recommendations[45][0]

In [23]:
utilities.retrieve_recommendations(train, top_ns_45)

For user: A3C2ECIXEQ0YFQ:
Purchase History:
             asin                                              title
30751  B0009YS4P0  Nutri-Vet Hip &amp; Joint Extra Strength Chewa...
37067  B000IBRI2Y              Dog Dazer II Ultrasonic Dog Deterrent
40175  B000OV4VAU                        Nutri-Vet Alaska Salmon Oil
53623  B0029NQTI8  Pedigree Choice Cuts Variety Pack Lamb/Vegetab...

Recommending:

          asin                                              title
0   B0002ASM94  Nylabone Dura Chew Pooch Pacifier Chew Bone Va...
1   B00153RC9I  Halo Holistic Natural Dry Cat Food for Adult Cats
2   B0002AB9FS                                 Grizzly Salmon Oil
3   B0048Z73TY   DERMagic Shampoo Bar, 3.75 oz, Certified Organic
4   B009MKCJAI                                           KONG TUG
5   B0002ASC1M  Pureness Heavyweight Large Crock Pet Dish, 52-...
6   B0002AS5PU  Kaytee CritterTrail Fun-nels Tubes Accessories...
7   B0002AQ228  Prevue Hendryx Pet Products Wrought Iron Fligh...
8

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

In [24]:
cold_start_users = utilities.generate_cold_start_users(train)

In [25]:
for n in tuple(zip([10, 25, 30, 45], [top_ns_10, top_ns_25, top_ns_30, top_ns_45])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'UB-CF',
                                                    top_ns = cold_start_top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n[0],
                                                    mf_based = False))

The UB-CF has an average recall@10: 0.00590, average novelty@10: 0.94339
The UB-CF has an average recall@25: 0.01710, average novelty@25: 0.95841
The UB-CF has an average recall@30: 0.01879, average novelty@30: 0.96341
The UB-CF has an average recall@45: 0.02601, average novelty@45: 0.97154


# Generating Recommended Items DataFrame

In [26]:
max_recommendations = (utilities
                       .generate_recommendations_df(
                           train, 
                           n_recommendations, 
                           "UB-CF", mf_based=False))

In [27]:
max_recommendations

,reviewerID,item_rank,asin,algorithm,title
0,A04173782GDZSQ91AJ7OD,0,B00153RC9I,UB-CF,Halo Holistic Natural Dry Cat Food for Adult Cats
1,A04173782GDZSQ91AJ7OD,1,B0002YFSDI,UB-CF,Mendota Products Dog Slip Lead
2,A04173782GDZSQ91AJ7OD,2,B0048Z73TY,UB-CF,"DERMagic Shampoo Bar, 3.75 oz, Certified Organic"
3,A04173782GDZSQ91AJ7OD,3,B009MKCJAI,UB-CF,KONG TUG
4,A04173782GDZSQ91AJ7OD,4,B0002ASC1M,UB-CF,"Pureness Heavyweight Large Crock Pet Dish, 52-..."
...,...,...,...,...,...
798598,AZZYW4YOE1B6E,23,B001V9P1HG,UB-CF,"Pet Ringworm Treatment, Mange Mites- Soothes T..."
798599,AZZYW4YOE1B6E,24,B0002DJXJE,UB-CF,JW Pet Good Cuz Rubber Dog Toy
798600,AZZYW4YOE1B6E,25,B00CIBATOQ,UB-CF,Comfort Zone with Feliway Cats Diffuser and Ad...
798601,AZZYW4YOE1B6E,26,B003OYIAW4,UB-CF,Walky Dog Plus Hands Free Dog Bicycle Exercise...


# Store in `SQLite` DB

In [30]:
engine = create_engine("sqlite:///recommender.db", echo=True)

In [31]:
max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")

2021-09-28 18:05:33,502 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Pet_Supplies")
2021-09-28 18:05:33,503 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-28 18:05:33,504 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Pet_Supplies")
2021-09-28 18:05:33,505 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-28 18:05:33,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-28 18:05:33,509 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Pet_Supplies" (
	"index" BIGINT, 
	"reviewerID" TEXT, 
	item_rank BIGINT, 
	asin TEXT, 
	algorithm TEXT, 
	title TEXT
)


2021-09-28 18:05:33,509 INFO sqlalchemy.engine.Engine [no key 0.00065s] ()
2021-09-28 18:05:33,511 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Pet_Supplies_index" ON "Pet_Supplies" ("index")
2021-09-28 18:05:33,512 INFO sqlalchemy.engine.Engine [no key 0.00083s] ()
2021-09-28 18:05:33,514 INFO sqlalchemy.engine.Engine COMMIT
2021-09-28 18:05:33,736 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-28 18:05: